In [2]:
import pandas as pd
import requests

In [3]:
# Lista endpointów do pobrania
endpoints = {
    "launches": "https://api.spacexdata.com/v4/launches",
    "rockets": "https://api.spacexdata.com/v4/rockets",
    "launchpads": "https://api.spacexdata.com/v4/launchpads",
    "payloads": "https://api.spacexdata.com/v4/payloads",
    "cores": "https://api.spacexdata.com/v4/cores",
    "capsules": "https://api.spacexdata.com/v4/capsules",
    "crew": "https://api.spacexdata.com/v4/crew",
    "dragons": "https://api.spacexdata.com/v4/dragons"
}

# Słownik na DataFrame'y
dataframes = {}

for name, url in endpoints.items():
    print(f"Pobieranie danych z {name}...")
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data)
        dataframes[name] = df
        print(f"✓ Pobrano {len(df)} rekordów z {name}.")
    else:
        print(f"✗ Błąd pobierania {name}: {response.status_code}")

# Zapis do plików CSV (opcjonalnie)
for name, df in dataframes.items():
    df.to_csv(f"{name}.csv", index=False)

print("✅ Zakończono pobieranie wszystkich danych.")

Pobieranie danych z launches...
✓ Pobrano 205 rekordów z launches.
Pobieranie danych z rockets...
✓ Pobrano 4 rekordów z rockets.
Pobieranie danych z launchpads...
✓ Pobrano 6 rekordów z launchpads.
Pobieranie danych z payloads...
✓ Pobrano 225 rekordów z payloads.
Pobieranie danych z cores...
✓ Pobrano 83 rekordów z cores.
Pobieranie danych z capsules...
✓ Pobrano 25 rekordów z capsules.
Pobieranie danych z crew...
✓ Pobrano 30 rekordów z crew.
Pobieranie danych z dragons...
✓ Pobrano 2 rekordów z dragons.
✅ Zakończono pobieranie wszystkich danych.


In [5]:
from itertools import combinations

# Lista nazw DataFrame'ów
table_names = list(dataframes.keys())

# Sprawdzenie wspólnych kolumn między każdą parą
for t1, t2 in combinations(table_names, 2):
    common_cols = set(dataframes[t1].columns) & set(dataframes[t2].columns)
    if common_cols:
        print(f"Wspólne kolumny między {t1} i {t2}: {common_cols}")

Wspólne kolumny między launches i rockets: {'id', 'name'}
Wspólne kolumny między launches i launchpads: {'id', 'name', 'details'}
Wspólne kolumny między launches i payloads: {'id', 'name'}
Wspólne kolumny między launches i cores: {'id'}
Wspólne kolumny między launches i capsules: {'id'}
Wspólne kolumny między launches i crew: {'id', 'name'}
Wspólne kolumny między launches i dragons: {'id', 'name'}
Wspólne kolumny między rockets i launchpads: {'id', 'name'}
Wspólne kolumny między rockets i payloads: {'id', 'name', 'type'}
Wspólne kolumny między rockets i cores: {'id'}
Wspólne kolumny między rockets i capsules: {'id', 'type'}
Wspólne kolumny między rockets i crew: {'wikipedia', 'name', 'id'}
Wspólne kolumny między rockets i dragons: {'diameter.feet', 'type', 'flickr_images', 'wikipedia', 'active', 'id', 'diameter.meters', 'name', 'first_flight', 'description'}
Wspólne kolumny między launchpads i payloads: {'id', 'name', 'longitude'}
Wspólne kolumny między launchpads i cores: {'id', 'stat

In [6]:
# Sprawdzenie kolumn zawierających ID (nazwa kolumny = rocket, launchpad, itp.)
for name, df in dataframes.items():
    id_like_cols = [col for col in df.columns if 'id' in col or 'rocket' in col or 'capsule' in col or 'launchpad' in col or 'payload' in col]
    print(f"{name}: {id_like_cols}")

launches: ['rocket', 'capsules', 'payloads', 'launchpad', 'launch_library_id', 'id', 'links.youtube_id']
rockets: ['payload_weights', 'id', 'second_stage.payloads.composite_fairing.height.meters', 'second_stage.payloads.composite_fairing.height.feet', 'second_stage.payloads.composite_fairing.diameter.meters', 'second_stage.payloads.composite_fairing.diameter.feet', 'second_stage.payloads.option_1']
launchpads: ['rockets', 'id']
payloads: ['norad_ids', 'id', 'dragon.capsule']
cores: ['id']
capsules: ['id']
crew: ['id']
dragons: ['sidewall_angle_deg', 'id', 'launch_payload_mass.kg', 'launch_payload_mass.lb', 'launch_payload_vol.cubic_meters', 'launch_payload_vol.cubic_feet', 'return_payload_mass.kg', 'return_payload_mass.lb', 'return_payload_vol.cubic_meters', 'return_payload_vol.cubic_feet', 'pressurized_capsule.payload_volume.cubic_meters', 'pressurized_capsule.payload_volume.cubic_feet']


In [8]:
dataframes["launches"].columns.tolist() 

['static_fire_date_utc',
 'static_fire_date_unix',
 'net',
 'window',
 'rocket',
 'success',
 'failures',
 'details',
 'crew',
 'ships',
 'capsules',
 'payloads',
 'launchpad',
 'flight_number',
 'name',
 'date_utc',
 'date_unix',
 'date_local',
 'date_precision',
 'upcoming',
 'cores',
 'auto_update',
 'tbd',
 'launch_library_id',
 'id',
 'fairings.reused',
 'fairings.recovery_attempt',
 'fairings.recovered',
 'fairings.ships',
 'links.patch.small',
 'links.patch.large',
 'links.reddit.campaign',
 'links.reddit.launch',
 'links.reddit.media',
 'links.reddit.recovery',
 'links.flickr.small',
 'links.flickr.original',
 'links.presskit',
 'links.webcast',
 'links.youtube_id',
 'links.article',
 'links.wikipedia',
 'fairings']

In [47]:
# czyszczenie danych dla launches

launches_raw = dataframes["launches"]

launches = launches_raw[[
    "id",           # potrzebne jako klucz główny
    "name",         # nazwa misji
    "date_utc",     # data startu
    "success",      # wynik startu
    "rocket",       # ID rakiety (połączysz z rockets)
    "launchpad",    # ID launchpada (dołączysz z launchpads)
    "payloads",     # lista ID – osobna tabela pomocnicza
    "crew",         # lista ID – j.w.
    "details",      # opis misji (opcjonalnie)
    "links.webcast"
]].copy()

# Konwersja na datetime
launches["date_utc"] = pd.to_datetime(launches["date_utc"], errors="coerce")

launches.head(10)

,id,name,date_utc,success,rocket,launchpad,payloads,crew,details,links.webcast
0,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24 22:30:00+00:00,False,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b5b6c3bb0006eeb1e1],[],Engine failure at 33 seconds and loss of vehicle,https://www.youtube.com/watch?v=0a_00nJ_Y88
1,5eb87cdaffd86e000604b32b,DemoSat,2007-03-21 01:10:00+00:00,False,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b6b6c3bb0006eeb1e2],[],Successful first stage burn and transition to ...,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc
2,5eb87cdbffd86e000604b32c,Trailblazer,2008-08-03 03:34:00+00:00,False,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",[],Residual stage 1 thrust led to collision betwe...,https://www.youtube.com/watch?v=v0w9p3U8860
3,5eb87cdbffd86e000604b32d,RatSat,2008-09-28 23:15:00+00:00,True,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e5],[],Ratsat was carried to orbit on the first succe...,https://www.youtube.com/watch?v=dLQ2tZEH6G0
4,5eb87cdcffd86e000604b32e,RazakSat,2009-07-13 03:35:00+00:00,True,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e6],[],None,https://www.youtube.com/watch?v=yTaIDooc8Og
5,5eb87cddffd86e000604b32f,Falcon 9 Test Flight,2010-06-04 18:45:00+00:00,True,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,[5eb0e4b7b6c3bb0006eeb1e7],[],None,https://www.youtube.com/watch?v=nxSxgBKlYws
6,5eb87cdeffd86e000604b330,COTS 1,2010-12-08 15:43:00+00:00,True,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,"[5eb0e4b9b6c3bb0006eeb1e8, 5eb0e4b9b6c3bb0006e...",[],None,https://www.youtube.com/watch?v=cdLITgWKe_0
7,5eb87cdfffd86e000604b331,COTS 2,2012-05-22 07:44:00+00:00,True,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,[5eb0e4bab6c3bb0006eeb1ea],[],"Launch was scrubbed on first attempt, second l...",https://www.youtube.com/watch?v=tpQzDbAY7yI
8,5eb87ce0ffd86e000604b332,CRS-1,2012-10-08 00:35:00+00:00,True,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,"[5eb0e4bab6c3bb0006eeb1eb, 5eb0e4bab6c3bb0006e...",[],"CRS-1 successful, but the secondary payload wa...",https://www.youtube.com/watch?v=-Vk3hiV_zXU
9,5eb87ce1ffd86e000604b333,CRS-2,2013-03-01 19:10:00+00:00,True,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84,[5eb0e4bbb6c3bb0006eeb1ed],[],Last launch of the original Falcon 9 v1.0 laun...,https://www.youtube.com/watch?v=ik0ElKl5kW4


In [48]:
rockets = dataframes["rockets"]
rocket_mapping = rockets.set_index("id")["name"].to_dict()

launches["rocket"] = launches["rocket"].map(rocket_mapping)

In [49]:
launchpads = dataframes["launchpads"]
launchpad_mapping = launchpads.set_index("id")["name"].to_dict()

launches["launchpad"] = launches["launchpad"].map(launchpad_mapping)

In [64]:
crew = dataframes["crew"]
crew_mapping = crew.set_index("id")["name"].to_dict()

launches["crew"] = launches["crew"].apply(
    lambda lst: ", ".join(crew_mapping.get(x, x) for x in lst) if isinstance(lst, list) else ""
)

launches["crew"] = launches["crew"].replace("", "Bezzałogowy")

In [ ]:
payloads = dataframes["payloads"]
payload_mapping = payloads.set_index("id")["name"].to_dict()

launches["payloads"] = launches["payloads"].apply(
    lambda lst: ", ".join(payload_mapping.get(x, x) for x in lst) if isinstance(lst, list) else ""
)

,id,name,date_utc,success,rocket,launchpad,payloads,crew,details,links.webcast
0,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24 22:30:00+00:00,False,Falcon 1,Kwajalein Atoll,FalconSAT-2,,Engine failure at 33 seconds and loss of vehicle,https://www.youtube.com/watch?v=0a_00nJ_Y88
1,5eb87cdaffd86e000604b32b,DemoSat,2007-03-21 01:10:00+00:00,False,Falcon 1,Kwajalein Atoll,DemoSAT,,Successful first stage burn and transition to ...,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc
2,5eb87cdbffd86e000604b32c,Trailblazer,2008-08-03 03:34:00+00:00,False,Falcon 1,Kwajalein Atoll,"Trailblazer, PRESat",,Residual stage 1 thrust led to collision betwe...,https://www.youtube.com/watch?v=v0w9p3U8860
3,5eb87cdbffd86e000604b32d,RatSat,2008-09-28 23:15:00+00:00,True,Falcon 1,Kwajalein Atoll,RatSat,,Ratsat was carried to orbit on the first succe...,https://www.youtube.com/watch?v=dLQ2tZEH6G0
4,5eb87cdcffd86e000604b32e,RazakSat,2009-07-13 03:35:00+00:00,True,Falcon 1,Kwajalein Atoll,RazakSAT,,None,https://www.youtube.com/watch?v=yTaIDooc8Og
5,5eb87cddffd86e000604b32f,Falcon 9 Test Flight,2010-06-04 18:45:00+00:00,True,Falcon 9,CCSFS SLC 40,Dragon Qualification Unit,,None,https://www.youtube.com/watch?v=nxSxgBKlYws
6,5eb87cdeffd86e000604b330,COTS 1,2010-12-08 15:43:00+00:00,True,Falcon 9,CCSFS SLC 40,"COTS Demo Flight 1, Cubesats",,None,https://www.youtube.com/watch?v=cdLITgWKe_0
7,5eb87cdfffd86e000604b331,COTS 2,2012-05-22 07:44:00+00:00,True,Falcon 9,CCSFS SLC 40,COTS Demo Flight 2,,"Launch was scrubbed on first attempt, second l...",https://www.youtube.com/watch?v=tpQzDbAY7yI
8,5eb87ce0ffd86e000604b332,CRS-1,2012-10-08 00:35:00+00:00,True,Falcon 9,CCSFS SLC 40,"CRS-1, Orbcomm-OG2",,"CRS-1 successful, but the secondary payload wa...",https://www.youtube.com/watch?v=-Vk3hiV_zXU
9,5eb87ce1ffd86e000604b333,CRS-2,2013-03-01 19:10:00+00:00,True,Falcon 9,CCSFS SLC 40,CRS-2,,Last launch of the original Falcon 9 v1.0 laun...,https://www.youtube.com/watch?v=ik0ElKl5kW4


In [60]:
launches = launches.rename(columns={"id": "original_launch_id"})
launches["launch_id"] = range(1, len(launches) + 1)
cols = ["launch_id"] + [col for col in launches.columns if col != "launch_id"]
launches = launches[cols]

In [65]:
launches.head(10)

,launch_id,original_launch_id,name,date_utc,success,rocket,launchpad,payloads,crew,details,links.webcast
0,1,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24 22:30:00+00:00,False,Falcon 1,Kwajalein Atoll,FalconSAT-2,Bezzałogowy,Engine failure at 33 seconds and loss of vehicle,https://www.youtube.com/watch?v=0a_00nJ_Y88
1,2,5eb87cdaffd86e000604b32b,DemoSat,2007-03-21 01:10:00+00:00,False,Falcon 1,Kwajalein Atoll,DemoSAT,Bezzałogowy,Successful first stage burn and transition to ...,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc
2,3,5eb87cdbffd86e000604b32c,Trailblazer,2008-08-03 03:34:00+00:00,False,Falcon 1,Kwajalein Atoll,"Trailblazer, PRESat",Bezzałogowy,Residual stage 1 thrust led to collision betwe...,https://www.youtube.com/watch?v=v0w9p3U8860
3,4,5eb87cdbffd86e000604b32d,RatSat,2008-09-28 23:15:00+00:00,True,Falcon 1,Kwajalein Atoll,RatSat,Bezzałogowy,Ratsat was carried to orbit on the first succe...,https://www.youtube.com/watch?v=dLQ2tZEH6G0
4,5,5eb87cdcffd86e000604b32e,RazakSat,2009-07-13 03:35:00+00:00,True,Falcon 1,Kwajalein Atoll,RazakSAT,Bezzałogowy,None,https://www.youtube.com/watch?v=yTaIDooc8Og
5,6,5eb87cddffd86e000604b32f,Falcon 9 Test Flight,2010-06-04 18:45:00+00:00,True,Falcon 9,CCSFS SLC 40,Dragon Qualification Unit,Bezzałogowy,None,https://www.youtube.com/watch?v=nxSxgBKlYws
6,7,5eb87cdeffd86e000604b330,COTS 1,2010-12-08 15:43:00+00:00,True,Falcon 9,CCSFS SLC 40,"COTS Demo Flight 1, Cubesats",Bezzałogowy,None,https://www.youtube.com/watch?v=cdLITgWKe_0
7,8,5eb87cdfffd86e000604b331,COTS 2,2012-05-22 07:44:00+00:00,True,Falcon 9,CCSFS SLC 40,COTS Demo Flight 2,Bezzałogowy,"Launch was scrubbed on first attempt, second l...",https://www.youtube.com/watch?v=tpQzDbAY7yI
8,9,5eb87ce0ffd86e000604b332,CRS-1,2012-10-08 00:35:00+00:00,True,Falcon 9,CCSFS SLC 40,"CRS-1, Orbcomm-OG2",Bezzałogowy,"CRS-1 successful, but the secondary payload wa...",https://www.youtube.com/watch?v=-Vk3hiV_zXU
9,10,5eb87ce1ffd86e000604b333,CRS-2,2013-03-01 19:10:00+00:00,True,Falcon 9,CCSFS SLC 40,CRS-2,Bezzałogowy,Last launch of the original Falcon 9 v1.0 laun...,https://www.youtube.com/watch?v=ik0ElKl5kW4
